# Robomimic Dynamic Movement Primitives (DMP) - Instructional Notebook

This notebook is an implementation of **Dynamic Movement Primitives (DMP)** within a `robomimic` simulation environment. The implementation is built on top of [robomimic tutorial](https://colab.research.google.com/drive/1b62r_km9pP40fKF0cBdpdTO2P_2eIbC6?usp=sharing).

---

## Objectives

- Understand the basics of motion planning with DMP.
- Integrate DMP into `robomimic`.
- Visualize planned trajectories and identify potential issues.
- Modify or extend the implementation as an exercise.

---

## Table of Contents

1. Environment Setup
2. Download Demonstration Dataset
3. Implement DMP Algorithm
4. Path Execution (Rollout)
5. Path Visualization

---

**Note**: Please run each cell in order. Some cells are left intentionally editable for hands-on experimentation.


### 1. Environment Setup

In [1]:
WS_DIR = "/content/"
%cd $WS_DIR

# Clone the repo and install the basic requirements
!git clone --branch v0.4 https://github.com/ARISE-Initiative/robomimic
!pip install -e robomimic/ > /dev/null

import sys
import os
sys.path.append('./robomimic/')

/content
Cloning into 'robomimic'...
remote: Enumerating objects: 3540, done.
remote: Counting objects: 100% (1625/1625), done.
remote: Compressing objects: 100% (361/361), done.
remote: Total 3540 (delta 1425), reused 1264 (delta 1264), pack-reused 1915 (from 4)
Receiving objects: 100% (3540/3540), 62.08 MiB | 15.32 MiB/s, done.
Resolving deltas: 100% (2456/2456), done.


In [2]:
# install all system dependencies for mujoco-py
!sudo DEBIAN_FRONTEND=noninteractive apt install curl git libgl1-mesa-dev libgl1-mesa-glx libglew-dev \
         libosmesa6-dev software-properties-common net-tools unzip vim \
         virtualenv wget xpra xserver-xorg-dev libglfw3-dev patchelf > /dev/null

#install mujoco-py
!pip install mujoco > /dev/null

#install robosuite
!pip install robosuite > /dev/null



Extracting templates from packages: 100%


## 2. Download Demonstration Dataset

In this step, we download the **robomimic demonstration dataset** for the `"can"` task.  
This dataset contains low-dimensional proprioceptive and object state information recorded from expert demonstrations.

We will later use this dataset not only to inspect robot and object configurations,  
but also to **use demonstration data** such as:

- Robot end-effector position (`robot0_eef_pos`)
- Orientation (`robot0_eef_quat`)
- Object states (`object`)

These demonstrations are essential for generating trajectories with DMP.

> **Note:**  
> - In Colab, rendering with MuJoCo requires `osmesa` or `egl`.  
> - This code sets `MUJOCO_GL` to `'osmesa'` by default for compatibility.


In [3]:
# import all utility functions

import os
import json
import h5py
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from scipy.spatial.transform import Rotation as R

import robomimic
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.test_utils as TestUtils
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.train_utils as TrainUtils
from robomimic.utils.dataset import SequenceDataset

from robomimic.config import config_factory
from robomimic.algo import algo_factory

# for rendering mujoco in colab, you need turn on egl
os.environ['MUJOCO_GL'] = 'osmesa' # if gpu possible, use 'egl', if not, use 'osmesa'

# the dataset registry can be found at robomimic/__init__.py
from robomimic import DATASET_REGISTRY, HF_REPO_ID

# set download folder and make it
download_folder = WS_DIR + "robomimic_data/"
os.makedirs(download_folder, exist_ok=True)

# download the dataset
task = "can"
dataset_type = "ph"
hdf5_type = "low_dim"
FileUtils.download_file_from_hf(
    repo_id=HF_REPO_ID,
    filename=DATASET_REGISTRY[task][dataset_type][hdf5_type]["url"],
    download_dir=download_folder,
)

# enforce that the dataset exists
dataset_path = os.path.join(download_folder, "low_dim_v15.hdf5")
assert os.path.exists(dataset_path)

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /content/./robomimic/robomimic/scripts/setup_macros.py
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


low_dim_v15.hdf5:   0%|          | 0.00/46.9M [00:00<?, ?B/s]

Visualize Playback Trajectory

- Replays 5 demonstrations episode from the dataset.

In [4]:
!python robomimic/robomimic/scripts/playback_dataset.py --dataset robomimic_data/low_dim_v15.hdf5 --video_path demo_playback_dataset.mp4 --video_skip 1 --n 5

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /content/robomimic/robomimic/scripts/setup_macros.py
)

============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_pos']
using obs modality: rgb with keys: []
[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /usr/local/lib/python3.11/dist-packages/robosuite/scripts/setup_macros.py (macros.py:55)
[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)
[robosuite WARNING] Could not load the mink-based whole-body IK. Make sure you install related import properl

In [5]:
# visualize rollout video

from IPython.display import HTML
from base64 import b64encode

mp4 = open("demo_playback_dataset.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")

Here, we analyze demonstrations to extract key information needed for DMP planning:

- Extract end-effector position (`eef_pos`), orientation (`quat`), and object states (`obj`).
- Compute the acceleration of gripper.
- Use acceleration of gripper to identify:
  - **Reach phase end**: when gripper first gets close to object.
  - **Grasp phase end**: when gripper starts moving away after grasp.
- Store target positions and orientations at key frames such as:
  - `reach_final_relative_pos` – object-relative configuration before grasp.
  - `grasp_goal_pos` – absolute robot configuration at grasp moment.
  - `finish_goal_pos` – absolute robot configuration at episode end.

> Object-relative information is useful for learning general grasping strategies,  
> while absolute-value information help guide the DMP in real-world coordinates.


In [6]:
def decided_phase(data):
    gripper_acc = np.diff(np.diff(data['obs/robot0_gripper_qpos']), axis=0)

    reach_end = np.argmax(gripper_acc > 5e-4)
    grasp_end = reach_end + np.argmax(gripper_acc[reach_end:] < -5e-4)

    return reach_end, grasp_end

f = h5py.File(dataset_path, "r")
dataset = f["data"]

reach_dataset = []
grasp_dataset = []
finish_dataset = []
keyframe_infos = []
for i in dataset.keys():
    eef_pos = np.array(dataset[i]['obs/robot0_eef_pos'])
    quat = np.array(dataset[i]['obs/robot0_eef_quat'])
    obj = np.array(dataset[i]['obs/object'])
    reach_end, grasp_end = decided_phase(dataset[i])

    reach_dataset.append({'eef_pos': eef_pos[:reach_end], 'quat': quat[:reach_end], 'obj': obj[:reach_end]})
    grasp_dataset.append({'eef_pos': eef_pos[reach_end:grasp_end], 'quat': quat[reach_end:grasp_end], 'obj': obj[reach_end:grasp_end]})
    finish_dataset.append({'eef_pos': eef_pos[grasp_end:], 'quat': quat[grasp_end:], 'obj': obj[grasp_end:]})

    keyframe_infos.append({
        'reach_goal_relative_pos': obj[reach_end-1, :3],
        'reach_goal_relative_quat': obj[reach_end-1, 3:7],
        'grasp_goal_pos': eef_pos[grasp_end-1],
        'grasp_goal_quat': quat[grasp_end-1],
        'finish_goal_pos': eef_pos[-1],
        'finish_goal_quat': quat[-1],
    })

reach_goal_relative_pos = np.mean(np.stack([d['reach_goal_relative_pos'] for d in keyframe_infos]), axis=0)
reach_goal_relative_quat = np.mean(np.stack([d['reach_goal_relative_quat'] for d in keyframe_infos]), axis=0)
grasp_goal_pos = np.mean(np.stack([d['grasp_goal_pos'] for d in keyframe_infos]), axis=0)
grasp_goal_quat = np.mean(np.stack([d['grasp_goal_quat'] for d in keyframe_infos]), axis=0)
finish_goal_pos = np.mean(np.stack([d['finish_goal_pos'] for d in keyframe_infos]), axis=0)
finish_goal_quat = np.mean(np.stack([d['finish_goal_quat'] for d in keyframe_infos]), axis=0)

## 3. Implement DMP Algorithm

We now implement the **Dynamic Movement Primitives (DMP)** framework to generate smooth, goal-directed trajectories from demonstrations.

### Key Concepts

* **Canonical System**:

  Acts as a phase variable that monotonically decays from 1 to 0 (for discrete movements).
  
  Governs temporal progression:
  $\tau\dot{s} = -\alpha s$
  
  This variable controls how basis functions are activated over time.

* **Transformation System**:

  Models a damped spring system augmented with a learnable forcing term:

  $$
  \tau \dot{v}=K(g-x)-Dv+(g-x_0)f
  $$
  
  $$
  \tau \dot{x}=v
  $$

  * \$x, v\$: current position and velocity
  * \$x_0, g\$: start and goal position
  * \$\tau\$: temporal scaling factor
  * \$K\$: acts like spring constant
  * \$D\$: damping term
  * \$f\$: forcing term

* **Forcing Term \$f(s)\$**:

  A weighted sum of radial basis functions (RBFs), modulated by the canonical variable:

  $$
  f(s) = \frac{\sum_i w_i\psi_i(s) s}{\sum_i \psi_i(s)}
  $$

  where each basis function \$\psi\_i(s) = \exp(-h\_i (s - c\_i)^2)\$.

* **Weight Learning**:

  From demonstrated trajectories, compute target acceleration and solve for weights \${w\_i}\$ using weighted linear regression.

---

### Adjustable Parameters

* `n_dmps`: number of dimensions (e.g., 3 for 3D position).
* `n_bfs`: number of basis functions (controls expressiveness).
* `dt`: timestep duration.
* `x0`, `goal`: initial and goal states.
* `K`, `D`: gain parameters controlling convergence speed.
* `tau`: scaling parameter for execution speed.
* `alpha`: canonical system parameter.
---

### Rollout Execution

After weight learning, the system simulates a trajectory with:

1. Phase variable update.
2. Basis function activation (`ψ_i(s)`).
3. Compute forcing term \$f(s)\$.
4. Integrate acceleration → velocity → position.

---

> ✅ The DMP system allows for **modulating**, **rescaling**, and **generalizing** trajectories with respect to new start/goal or execution speeds.

In [7]:
class CanonicalSystem:
    """Implementation of the canonical dynamical system
    as described in Dr. Stefan Schaal's (2002) paper"""

    def __init__(self, dt, alpha=1.0, tau=1.0):
        """Default values from Schaal (2012)"""
        self.alpha = alpha
        self.tau = tau

        self.run_time = 1.0

        self.dt = dt
        self.timesteps = int(self.run_time / self.dt)

        self.reset_state()

    def rollout(self, **kwargs):
        """Generate x for open loop movements.
        """
        timesteps = int(self.timesteps / self.tau)
        self.s_track = np.zeros(timesteps)

        self.reset_state()
        for t in range(timesteps):
            self.s_track[t] = self.s
            self.step()

        return self.s_track

    def reset_state(self):
        """Reset the system state"""
        self.s = 1.0

    def step(self):
        """Generate a single step of s for discrete
        (potentially closed) loop movements.
        Decaying from 1 to 0 according to ds = -alpha*s.

        tau float: gain on execution time
                   increase tau to make the system execute faster
        """
        self.s += (-self.alpha * self.s) * self.tau * self.dt
        return self.s

In [8]:
class TransformationSystem:
    """Implementation of the transformation dynamical system"""
    def __init__(self, K, D):
        self.K = K
        self.D = D

    def get_acc(self, y, dy, y0, g, tau, f, s):
        return (self.K * (g - y) - self.D * dy + (g - y0) * f) / tau

    def get_f(self, y, dy, ddy, y0, g, tau, s):
        return (tau**2 * ddy - self.K * (g - y) + tau * self.D * dy) / (g - y0)

class ModifiedTransformationSystem(TransformationSystem):
    """TODO: You need to implement modified transformation system in Schaal's paper (https://ieeexplore.ieee.org/document/5152385)"""
    def __init__(self, K, D):
        super().__init__(K, D)

    def get_acc(self, y, dy, y0, g, tau, f, s):
        return (self.K * (g - y) - self.D * dy - self.K * (g - y0) * s + self.K * f) / tau

    def get_f(self, y, dy, ddy, y0, g, tau, s):
        return (tau * ddy + self.D * dy) / self.K - (g - y) + (g - y0) * s

In [31]:
class DMPs(object):
    """Implementation of Dynamic Motor Primitives"""

    def __init__(
        self, n_dmps, n_bfs, dt=0.01, y0=0, goal=1, w=None, K=None, D=None, alpha=1.0, tau=1.0, **kwargs
    ):
        """
        n_dmps int: number of dynamic motor primitives
        n_bfs int: number of basis functions per DMP
        dt float: timestep for simulation
        y0 list: initial state of DMPs
        goal list: goal state of DMPs
        w list: tunable parameters, control amplitude of basis functions
        K int: gain on attractor term y dynamics
        D int: gain on attractor term y dynamics
        """

        self.n_dmps = n_dmps
        self.n_bfs = n_bfs
        self.dt = dt
        if isinstance(y0, (int, float)):
            y0 = np.ones(self.n_dmps) * y0
        self.y0 = y0
        if isinstance(goal, (int, float)):
            goal = np.ones(self.n_dmps) * goal
        self.goal = goal
        if w is None:
            # default is f = 0
            w = np.zeros((self.n_dmps, self.n_bfs))
        self.w = w
        self.w_list = []

        # set up the CS
        self.alpha = alpha
        self.tau = tau

        self.cs = CanonicalSystem(dt=self.dt, alpha=self.alpha, tau=self.tau)
        self.timesteps = int(self.cs.run_time / self.dt)

        self.K = 300 * self.tau if K is None else K
        self.D = 2 * np.sqrt(self.K * self.tau) if D is None else D
        self.transformation = TransformationSystem(K=self.K, D=self.D)

        # set up the DMP system
        self.reset_state()

        self.gen_centers()

        # set variance of Gaussian basis functions
        # trial and error to find this spacing
        self.h = np.ones(self.n_bfs) * self.n_bfs ** 1.5 / self.c / self.cs.alpha

        self.check_offset()

    def check_offset(self):
        """Check to see if initial position and goal are the same
        if they are, offset slightly so that the forcing term is not 0"""

        for d in range(self.n_dmps):
            if abs(self.y0[d] - self.goal[d]) < 1e-4:
                self.goal[d] += 1e-4

    def gen_centers(self):
        """Set the centre of the Gaussian basis
        functions be spaced evenly throughout run time"""

        # desired activations throughout time
        des_c = np.linspace(0, self.cs.run_time, self.n_bfs)

        self.c = np.ones(len(des_c))
        for n in range(len(des_c)):
            # finding x for desired times t
            self.c[n] = np.exp(-self.cs.alpha * des_c[n])

    def gen_psi(self, s):
        """Generates the activity of the basis functions for a given
        canonical system rollout.

        s float, array: the canonical system state or path
        """

        if isinstance(s, np.ndarray):
            s = s[:, None]
        return np.exp(-self.h * (s - self.c) ** 2)

    def gen_weights(self, f_target):
        """Generate a set of weights over the basis functions such
        that the target forcing term trajectory is matched.

        f_target np.array: the desired forcing term trajectory
        """

        # efficiently calculate BF weights using weighted linear regression
        w = np.zeros((self.n_dmps, self.n_bfs))

        # calculate s and psi
        s_track = self.cs.rollout()
        psi_track = self.gen_psi(s_track)
        for d in range(self.n_dmps):
            for b in range(self.n_bfs):
                numer = np.sum(psi_track[:, b] * f_target[:, d])
                denom = np.sum(psi_track[:, b] * s_track)
                w[d, b] = numer / denom

        self.w_list.append(w)
        self.w = np.mean(self.w_list, axis=0)

    def imitate_path(self, y_des):
        """Takes in a desired trajectory and generates the set of
        system parameters that best realize this path.

        y_des list/array: the desired trajectories of each DMP
                          should be shaped [n_trajs, run_time, n_dmps]
        """

        B = len(y_des)

        self.y_des = y_des
        s_track = self.cs.rollout()
        for i in range(B):
            y_des = self.y_des[i]
            T, D = y_des.shape

            # set initial state and goal
            self.y0 = y_des[0, :]
            self.goal = y_des[-1, :]

            # generate function to interpolate the desired trajectory
            import scipy.interpolate

            path = np.zeros((self.n_dmps, self.timesteps))
            x = np.linspace(0, self.cs.run_time, T)
            for d in range(self.n_dmps):
                path_gen = scipy.interpolate.interp1d(x, y_des[:, d])
                for t in range(self.timesteps):
                    path[d, t] = path_gen(t * self.dt)
            y_des = path

            # calculate velocity of y_des with central differences
            dy_des = np.gradient(y_des, axis=1) / self.dt

            # calculate acceleration of y_des with central differences
            ddy_des = np.gradient(dy_des, axis=1) / self.dt

            f_target = np.zeros((self.timesteps, self.n_dmps))
            # find the force required to move along this trajectory
            for d in range(self.n_dmps):
                f_target[:, d] = self.transformation.get_f(y_des[d, :], dy_des[d, :], ddy_des[d, :], self.y0[d], self.goal[d], self.tau, s_track)

            # efficiently generate weights to realize f_target
            self.gen_weights(f_target)

            self.reset_state()

    def rollout(self, timesteps=None, **kwargs):
        """Generate a system trial, no feedback is incorporated."""

        self.reset_state()

        if timesteps is None:
            timesteps = int(self.timesteps / self.tau)

        # set up tracking vectors
        y_track = np.zeros((timesteps, self.n_dmps))
        dy_track = np.zeros((timesteps, self.n_dmps))
        ddy_track = np.zeros((timesteps, self.n_dmps))

        for t in range(timesteps):

            # run and record timestep
            y_track[t], dy_track[t], ddy_track[t] = self.step(**kwargs)

        return y_track, dy_track, ddy_track

    def reset_state(self):
        """Reset the system state"""
        self.y = self.y0.copy()
        self.dy = np.zeros(self.n_dmps)
        self.ddy = np.zeros(self.n_dmps)
        self.cs.reset_state()

    def step(self, tau=1.0):
        """Run the DMP system for a single timestep.

        tau float: scales the timestep
                   increase tau to make the system execute faster
        """

        # run canonical system
        s = self.cs.step()

        # generate basis function activation
        psi = self.gen_psi(s)

        for d in range(self.n_dmps):

            # generate the forcing term
            f = s * (np.dot(psi, self.w[d]))
            sum_psi = np.sum(psi)
            if np.abs(sum_psi) > 1e-6:
                f /= sum_psi

            # DMP acceleration
            self.ddy[d] = self.transformation.get_acc(self.y[d], self.dy[d], self.y0[d], self.goal[d], self.tau, f, s)
            self.dy[d] += self.ddy[d] * tau * self.dt
            self.y[d] += self.dy[d] * tau * self.dt

        return self.y, self.dy, self.ddy

Then, we define DMP Utilities

- `learn_dmp_from_traj(eef_pos, quat, dt)`  
  → Learns a DMP over 7D trajectory (position + quaternion).
- `make_goal_from_obj(obj_pos, obj_quat, rel_pos, rel_quat)`  
  → Computes the absolute **goal pose** from an object pose and a relative grasp pose (optional).
- `generate_action(dmp, y0, goal, gripper)`  
  → Rolls out DMP from initial pose `y0` to `goal`, and converts it into:
  - Position delta (velocity-like) by finite differencing.
  - Rotation delta via quaternion-to-rotvec conversion.
  - Constant gripper signal.
- These functions modularize the process of:
  1. Learning from demonstrations.
  2. Goal adaptation.
  3. Action sequence generation.

In [32]:
def learn_dmp_from_traj(dataset, dt):
    traj = [np.concatenate([d['eef_pos'], d['quat']], axis=-1) for d in dataset]
    _, D = traj[0].shape

    dmp = DMPs(n_dmps=D, n_bfs=100, dt=dt, alpha=1.0)
    dmp.y0 = traj[0]
    dmp.goal = traj[-1]
    dmp.imitate_path(y_des=traj)

    return dmp

def make_goal_from_obj(obj_pos, obj_quat, rel_pos=None, rel_quat=None):
    goal_pos = obj_pos
    if rel_pos is not None:
        goal_pos = obj_pos + rel_pos

    r_obj = R.from_quat(obj_quat)
    r_goal = r_obj
    if rel_quat is not None:
        r_rel = R.from_quat(rel_quat)
        r_goal = r_rel * r_obj
    goal_quat = r_goal.as_quat()

    return np.concatenate([goal_pos, goal_quat], axis=-1)

def generate_action(dmp, y0, goal, gripper):
    dmp.y0 = y0
    dmp.goal = goal

    traj_gen, _, _ = dmp.rollout()
    T, D = traj_gen.shape

    action_scale_modifier = 80
    act_pos = np.diff(traj_gen[:, :3], axis=0) * action_scale_modifier

    cur_rot = R.from_quat(traj_gen[:-1, 3:7])
    next_rot = R.from_quat(traj_gen[1:, 3:7])
    act_quat = (next_rot * cur_rot.inv()).as_rotvec()

    act_gripper = np.full((T-1, 1), gripper)

    actions = np.concatenate([act_pos, act_quat, act_gripper], axis=-1)

    return actions

Here, we make segment-wise DMPs from demonstration.

- Learn DMPs for each motion phase:
  - `reach`: from start to contact
  - `grasp`: close gripper and transport
  - `finish`: release


In [33]:
reach_dmp = learn_dmp_from_traj(reach_dataset, dt=1/60)
grasp_dmp = learn_dmp_from_traj(grasp_dataset, dt=1/60)
finish_dmp = learn_dmp_from_traj(finish_dataset, dt=1/20)

## 4. Path Execution (Rollout)

Now that we've implemented DMP, we will **execute actions in the robomimic**.

### Environment Setup

We first create a robomimic-compatible environment from metadata in the dataset.  
This ensures that the simulation settings (robot type, object type, etc.) exactly match the demonstration data.

In [34]:
import robomimic.utils.env_utils as EnvUtils

dummy_spec = dict(
    obs=dict(
        low_dim=["robot0_eef_pos", "robot0_eef_quat", "robot0_gripper_qpos", "object"],
        rgb=[],
    ),
)
ObsUtils.initialize_obs_utils_with_obs_specs(obs_modality_specs=dummy_spec)

env_meta = FileUtils.get_env_metadata_from_dataset(dataset_path)

env = EnvUtils.create_env_from_metadata(
    env_meta=env_meta,
    env_name=env_meta["env_name"],
    render=False,
    render_offscreen=True,
    use_image_obs=False,
)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_gripper_qpos', 'robot0_eef_pos', 'robot0_eef_quat', 'object']
using obs modality: rgb with keys: []
Created environment with name PickPlaceCan
Action size is 7


### Rollout Function

We define a `rollout` function that:
- Takes in the environment and a sequence of predicted actions
- Executes the actions in the simulation, one step at a time
- Records frames for visualization
- Tracks success metrics such as task completion and total reward


In [35]:
def rollout(env, pred_actions, video_writer):
    results = {}

    video_frames = []
    video_count = 0  # video frame counter

    rews = []
    success = None # success metrics

    for ac in pred_actions:
        # play action
        ob_dict, r, done, _ = env.step(ac)

        # compute reward
        rews.append(r)

        cur_success_metrics = env.is_success()

        if success is None:
            success = deepcopy(cur_success_metrics)
        else:
            for k in success:
                success[k] = success[k] | cur_success_metrics[k]

        # visualization
        frame = env.render(mode="rgb_array", height=512, width=512)
        video_frames.append(frame)
        video_count += 1

        # break if done
        if done:
            end_step = video_count
            break

    for frame in video_frames:
        video_writer.append_data(frame)

    end_step = video_count
    total_reward = np.sum(rews[:end_step + 1])

    results["Return"] = total_reward
    results["Horizon"] = end_step + 1
    results["Success_Rate"] = float(success["task"])

    # log additional success metrics
    for k in success:
        if k != "task":
            results["{}_Success_Rate".format(k)] = float(success[k])

    return ob_dict, results

###Action Execution Steps

We execute **three DMP rollouts**, corresponding to stages in a typical manipulation task:

1. **Reach the Object**
   - Plan a path from the robot's current end-effector position to the object.
   - Use DMP to generate the trajectory and convert it to a list of actions.
   - Execute the actions with gripper open (`-1`).

2. **Grasp and Transport**
   - Plan a new path from the grasp point to a transport goal position.
   - Execute with gripper closed (`1`).

3. **Arrive and Release**
   - Execute with gripper open (`-1`) to release.
   
   ---

> 📌 Note:  
> All actions are formatted as `[x, y, z, qx, qy, qz, gripper]` where `gripper` is `-1` (open) or `1` (close).


In [36]:
def rollout_phase(phase, dmp, ob_dict, video_writer):
    y0 = np.concatenate([ob_dict["robot0_eef_pos"], ob_dict["robot0_eef_quat"]], axis=-1)
    gripper = -1
    if phase == "reach":
        goal = make_goal_from_obj(ob_dict["object"][7:10], ob_dict["object"][10:], reach_goal_relative_pos, reach_goal_relative_quat)
    elif phase == "grasp":
        goal = make_goal_from_obj(grasp_goal_pos, grasp_goal_quat)
        gripper = 1
    elif phase == "finish":
        goal = make_goal_from_obj(finish_goal_pos, finish_goal_quat)

    pred_actions = generate_action(dmp, y0, goal, gripper=gripper)
    ob_dict, results = rollout(env, pred_actions, video_writer)
    return ob_dict, results

In [39]:
from robomimic.algo import RolloutPolicy
import imageio
from copy import deepcopy

# create a video writer
video_path = "rollout.mp4"
video_writer = imageio.get_writer(video_path, fps=20)

env.reset()
ob_dict, _, _, _ = env.step(np.zeros(7))

ob_dict, results = rollout_phase("reach", reach_dmp, ob_dict, video_writer)
ob_dict, results = rollout_phase("grasp", grasp_dmp, ob_dict, video_writer)
ob_dict, results = rollout_phase("finish", finish_dmp, ob_dict, video_writer)

video_writer.close()

## 5. Path Visualization

### Video Logging

- A video of the entire trajectory is saved as `rollout.mp4`.

In [40]:
# visualize rollout video

from IPython.display import HTML
from base64 import b64encode

mp4 = open(video_path, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")